# Ranking embedding

In [1]:
import numba 
import numpy as np

In [2]:
np.random.seed(0)

The problem consists of ranking $m$ items. Let's suppose those items are denotes from $1$ to $m$.

We consider $Y = \mathfrak{S}_m$, 
for $\sigma\in Y$, $\sigma(i) > \sigma(j)$ means that item $i$ is prefered over item $j$.

We represent $\sigma\in Y$ with $s = [\sigma(1), \cdots, \sigma(m)]$.
This representation can be thought as a canonical score associated to $\sigma\in Y$. 
If $s(i)$ is the score of item $i$, $s(i) > s(j)$ means that $i$ is prefered over $j$.

In [3]:
m = 10
sigma = np.random.permutation(m)
print(sigma)

[2 8 4 9 1 6 7 3 0 5]


We can derive a ranking represention $[\sigma^{-1}(1), \cdots, \sigma^{-1}(m)]$ from the scoring one with an argsort.

In [4]:
rank = sigma.argsort()
print(rank)

[8 4 0 7 2 9 5 6 1 3]


This argsort allows to recover also the canonical representation from the rank one in the same way.

In [5]:
print(rank.argsort())

[2 8 4 9 1 6 7 3 0 5]


Let's now switch to the Kemeny embedding to represent permutation, that is
$$
        \phi(\sigma) = (\text{sign}(\sigma(i) - \sigma(j)))
$$
To represent it, we will use the following indice mapping

 ind | j=1 | j=2 | j=3 | j=4
 --- | --- | --- | --- | ---
 i=1 |  *  |  1  |  2  |  4 
 i=2 |  *  |  *  |  3  |  5 
 i=3 |  *  |  *  |  *  |  6 
 i=4 |  *  |  *  |  *  |  * 

With the flat embedding, only based on pairs for which $i < j$
$$
   \text{emb}(\text{ind}(i, j)) = \phi(\sigma)_{ij}.
$$
To ease some calculation we will also use the symmetric embedding
$$
   \text{sym emb}(i, j) = \phi(\sigma)_{ij}.
$$

In [6]:
@numba.jit("i8[:, :](i8)", nopython=True)
def canonical_map(m):
    ind_map = np.full((m, m), m**2, dtype=np.int64)
    ind = 0
    for j in range(m):
        for i in range(j):
            ind_map[i, j] = ind
            ind += 1
    return ind_map


def get_emb(scores, ind_map):
    m = len(scores)
    m_emb = (m*(m-1))//2
    emb = np.empty(m_emb, dtype=np.float64)
    if scores.dtype == np.int64:
        fill_emb_i8(scores, emb, ind_map)
    else:
        fill_emb_f8(scores, emb, ind_map)
    return emb


@numba.jit("(i8[:], f8[:], i8[:, :])", nopython=True)
def fill_emb_i8(scores, emb, ind_map):
    m = len(ind_map)
    for j in range(m):
        for i in range(j):
            emb[ind_map[i, j]] = scores[i] > scores[j]
    emb *= 2
    emb -= 1


@numba.jit("(f8[:], f8[:], i8[:, :])", nopython=True)
def fill_emb_f8(scores, emb, ind_map):
    m = len(ind_map)
    for j in range(m):
        for i in range(j):
            emb[ind_map[i, j]] = scores[i] > scores[j]
    emb *= 2
    emb -= 1


def get_emb_from_rank(rank, ind_map):
    m = len(ind_map)
    m_emb = (m*(m-1)) // 2
    emb = np.zeros(m_emb, dtype=np.float64)
    fill_emb_from_rank(rank, emb, ind_map)
    return emb


@numba.jit("(i8[:], f8[:], i8[:, :])", nopython=True)
def fill_emb_from_rank(rank, emb, ind_map):
    for i_, i in enumerate(rank):
        for j in rank[i_+1:]:
            if i < j:
                ind = ind_map[i, j]
                emb[ind] = -1
            if j < i:
                ind = ind_map[j, i]
                emb[ind] = 1


def get_sym_emb(emb, ind_map):
    m = len(ind_map)
    sym_emb = np.zeros((m, m), dtype=np.float64)
    fill_sym_emb(emb, sym_emb, ind_map)
    return sym_emb


@numba.jit("(f8[:], f8[:, :], i8[:, :])", nopython=True)
def fill_sym_emb(emb, sym_emb, ind_map):
    m = len(ind_map)
    for j in range(m):
        sym_emb[j, j] = 0
        for i in range(j):
            ind = ind_map[i, j]
            sym_emb[i, j] = emb[ind]
            sym_emb[j, i] = -emb[ind]

In [7]:
ind_map = canonical_map(m)
emb = get_emb(sigma, ind_map)
print(np.abs(get_emb_from_rank(rank, ind_map) - emb).max())

0.0


The symmetric embedding allows to recover easily a score, as the number of times, item $i$ was bigger than an item $j$ minus the number of time it was smaller.

In [8]:
sym_emb = get_sym_emb(emb, ind_map)
new_scores = sym_emb.sum(axis=1)
print(new_scores.argsort().argsort())

[2 8 4 9 1 6 7 3 0 5]


In [9]:
new_emb = get_emb(new_scores, ind_map)
print(np.abs(new_emb - emb).max())

0.0


## A basic FAS solver

The precedent development allows to create an easy minimum feedback arc set solver.
We will work directly with the Kemeny embedding.

In [10]:
class BasicFasSolver:
    def __init__(self, ind_map):
        self.ind_map = ind_map
        
        # Placeholders
        m = len(ind_map)
        self.sym_pl = np.empty((m, m), dtype=np.float)
        self.score_pl = np.empty(m, dtype=np.float)
    
    def solve(self, c):
        """
        Solve inf_y <phi(y), c>.
        """
        emb = np.empty(c.shape, dtype=np.float)
        self.solve_out(c, emb)
        return emb
    
    def solve_out(self, c, out):
        fill_sym_emb(c, self.sym_pl, self.ind_map)
        np.sum(self.sym_pl, axis=1, out=self.score_pl) 
        self.score_pl *= -1
        fill_emb_f8(self.score_pl, out, self.ind_map)

In [11]:
solver = BasicFasSolver(ind_map)

In [12]:
print(np.abs(solver.solve(emb) + emb).max())

0.0


## Some function to gain time
Just-in-time compilation allows to gain a lot of computation time in python. A lot of the embedding function are going to be called on a series of entry, implying a for loop, to accelerate it, it is nice to have it in JIT. Therefore we introduce those functions.

In [13]:
def get_sym_emb(emb, ind_map):
    m = len(ind_map)
    sym_emb = np.zeros((m, m), dtype=np.float64)
    fill_sym_emb(emb, sym_emb, ind_map)
    return sym_emb


@numba.jit("(f8[:], f8[:, :], i8[:, :])", nopython=True)
def fill_sym_emb(emb, sym_emb, ind_map):
    m = len(ind_map)
    for j in range(m):
        sym_emb[j, j] = 0
        for i in range(j):
            ind = ind_map[i, j]
            sym_emb[i, j] = emb[ind]
            sym_emb[j, i] = -emb[ind]